In [24]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import time
from multiprocessing import Pool
import get_content

In [25]:
df=pd.read_csv('서울시 카페.csv')

In [26]:
urls = []
for i in range(len(df)):
    row = df.loc[i]
    y_axis = row['latitude']
    x_axis = row['longitude']
    url = f'https://www.nicebizmap.co.kr/util/readData.jsp?LF=nice.Analyse&LID=getMapControl&sql_type=flowpop&x_axis={x_axis}&y_axis={y_axis}&radius=250&readType=json'
    urls.append(url)

In [27]:
len(urls)

7359

In [5]:
if __name__ ==  '__main__':
    start_time = time.time()
    num_processors = 8
    p=Pool(processes = num_processors)
    output = p.map(get_content.get_content,urls)
    print("--- elapsed time %s seconds ---" % (time.time() - start_time))

--- elapsed time 1897.2851548194885 seconds ---


In [6]:
len(output)

7359

In [29]:
flow_pop = []

In [30]:
for i in tqdm(output):
    try:
        result = []
        for flow in i:
            x=float(flow['x_axis'])-float(x_axis)
            y=float(flow['y_axis'])-float(y_axis)
            result.append([abs(x)+abs(y),flow['flow_pop']])

        result.sort()
        result=result[0][1]
        flow_pop.append(result)  
    except:
        flow_pop.append("NaN")
        continue

100%|█████████████████████████████████████████████████████████████████████████████| 7359/7359 [00:57<00:00, 127.45it/s]


In [31]:
len(flow_pop)

7359

In [32]:
df_sub=pd.DataFrame({'flow_pop': flow_pop})
df=pd.concat([df,df_sub],axis=1)

In [33]:
df = df.drop(['Unnamed: 0'],axis=1)

In [49]:
df = df.loc[df['flow_pop'] != "NaN"]

In [50]:
df.to_csv('서울시 카페 유동인구.csv', encoding='utf-8-sig' ,index=False)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7359 entries, 0 to 7358
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                7359 non-null   object 
 1   latitude            7359 non-null   float64
 2   longitude           7359 non-null   float64
 3   address             7358 non-null   object 
 4   review              7329 non-null   object 
 5   savecount           7359 non-null   object 
 6   visitorreviewscore  5524 non-null   float64
 7   flow_pop            7359 non-null   object 
dtypes: float64(3), object(5)
memory usage: 460.1+ KB
